# Pryngles Tutorials: Developers
## Part 3 - Module Spangler

This notebook illustrate in detail the interface of the `Pryngles` package. It is especially intended for developers.

Authors:
- Jorge I. Zuluaga, [jorge.zuluaga@udea.edu.co](mailto:jorge.zuluaga@udea.edu.co)

Warnings:
- This file was prepared using version 0.7.3.1 of `Pryngles`.
- Matplotlib commands run in version 3.5.2.

<center><font color='red' size='8'><i>This tutorial is under construction.</i></font></center>

## Preparation

Before running this tutorial you need to install `Pryngles`:

In [ ]:
"""
!python -m pip install -qU pryngles
!pip show pryngles
#""";

If you are running this tutrial in `Google Colab` please run the following lines:

In [ ]:
"""
import matplotlib.pyplot as plt
RESOLUTION = 150
%matplotlib inline
plt.rcParams['figure.dpi'] = RESOLUTION
# Sometimes you need to run twice this magic to be sure the backend is loaded
%matplotlib inline
#""";

## Import useful packages

In [ ]:
%load_ext autoreload
%autoreload 2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

Although it is not a good practice to import everything of a package, for this tutorial and to avoid excesively large names we will import everything from Pryngles:

In [ ]:
from pryngles import *

## Core modules

## Spangler (`spangler`)

This is one of the key modules of `Pryngles`.  Sampling the surface of a body (star, planet or ring) is just the beginning.  In order to compute all the effects for which `Pryngles` was designed we need to build a special object called a `Spangler`.

You may create the most simple `Spangler` using:

In [ ]:
sg=Spangler()

The most important attribute of a `Spangler` is a DataFrame containing the positions and orientations of spangles:

In [ ]:
print_df(sg.data)

These are the columns in the spangler DataFrame:

In [ ]:
sg.data.columns

In order to know the meaning of the columns see:

In [ ]:
print(SPANGLER_COLUMNS_DOC)

It is important to point out that creating a Spangler only creates the basic structure, the scafold of the Spangler:

In [ ]:
sg=Spangler(nspangles=1000)
print_df(sg.data.head(5))

In order to fill the spangler with actual positions you should use:

In [ ]:
sg.populate_spangler(shape="circle")
print_df(sg.data.head(5))

To check use the `plot3d` method:

In [ ]:
sg.plot3d()

or the two dimensional version:

In [ ]:
sg.plot2d(coords="ecl")

You may join several spanglers in a single one:

In [ ]:
nspangles=500

sg1=Spangler(nspangles=nspangles)
sg1.populate_spangler(shape="sphere",spangle_type=SPANGLE_SOLID_ROCK)

sg2=Spangler(nspangles=nspangles,n_equ=Science.direction(0,20))
sg2.populate_spangler(shape="ring",scale=3,ri=2/3,spangle_type=SPANGLE_GRANULAR)

sgj=Spangler(spanglers=[sg1,sg2])
sgj.plot3d(coords="ecl")

Notice that we have defined the type of spangles to be used.  The type of spangles can be used to differentiate spangles according to their properties.  Type of spangles are:

In [ ]:
[f"{spangle} = {eval(spangle)}" for spangle in Consts.get_all() if ('SPANGLE_' in spangle) and ('COLORS' not in spangle)]

In graphical representation, each type of spangle are of a different color.  Additionally, for computing the state of the spangle (visible, illuminated, transmitting) the following special spangles are considered:

In [ ]:
SPANGLES_SEMITRANSPARENT

You may transform the coordinates of the spangles to any arbitrary system using `set_intersect`:

In [ ]:
output=sgj.set_intersect(nvec=[0,0,1],center=[2,0,0])

Here `center` is the position of the origin of the intersection system.  In this case, the distance to the intersection of each spangle is calculated:

In [ ]:
sgj.data.d_int

You may check how the points looks like:

In [ ]:
sgj.plot3d(coords="int")

You may plot the location seing from the +z-axis:

In [ ]:
sgj.plot2d(coords="int")

When you calculate the coordinates of a `Spangler` in an *intersection* system, you may also calculate the convex hulls corresponding to the objects:

In [ ]:
sgj._calc_qhulls()
sgj._plot_qhulls()

Here for instance the convex hull of the ring is blocking some of the spangles of the planet and viceversa.

There are two methods using the power of the intersection algorithms:

In [ ]:
#Observer is in the direction of the z-axis
sgj.set_observer(nvec=Science.direction(0,90))
#Light source is over the x-y plane
sgj.set_luz(nvec=Science.direction(135,0))
sgj.plot2d(coords="obs")

You may see that the size and luminosity of the points in this representation are proportional to the angle between the direction of the light-source and the direction of the observer.  The rings are barely visible because they receive light by transmission.

The angles between the normal to the spangles and the direction of the observer and the source of light are in the following columns of the Spangler DataFrame:

In [ ]:
sgj.data[["cos_obs","cos_luz"]]

The state of the spangles can be checked in the following columns:

In [ ]:
sgj.data[list(SPANGLER_SOURCE_STATES)+list(SPANGLER_VISIBILITY_STATES)]

You may also check the states of the spangles using:

In [ ]:
sgj.plot3d(coords="ecl",statemark=0.05)

You may illuminate independently two objects

In [ ]:
sg1=Spangler(name="Planet 1",nspangles=100,center_equ=[-5,0,0])
sg1.populate_spangler(shape="sphere",spangle_type=SPANGLE_LIQUID,preset=True,scale=3)

sg2=Spangler(name="Planet 2",nspangles=100,center_equ=[+5,0,0])
sg2.populate_spangler(shape="sphere",spangle_type=SPANGLE_SOLID_ROCK,preset=True,scale=3)

sg=Spangler(spanglers=[sg1,sg2])

sg.set_observer([0,1,0])
sg.set_luz(nvec=[1,0,0],center=[0,0,0],name="Planet 1")
sg.set_luz(nvec=[-1,0,0],center=[0,0,0],name="Planet 2")

sg.plot3d()

The most interesting method if the `Spangler` module is that calculating the intersections between bodies.

In [ ]:
sps=[]
sg=Spangler(nspangles=nspangles,name="Parent",n_equ=[0,0,1],center_equ=[-7,0,0])
sg.populate_spangler(shape="sphere",spangle_type=SPANGLE_STELLAR,scale=3,seed=1,preset=1)
sps+=[sg]
sg=Spangler(nspangles=nspangles,name="Ring",n_equ=[1,0,0])
sg.populate_spangler(shape="ring",spangle_type=SPANGLE_GRANULAR,scale=2.5,seed=1,ri=1.5/2.5,boundary=0)
sps+=[sg]
sg=Spangler(nspangles=nspangles,name="Planet",n_equ=[0,0,1])
sg.populate_spangler(shape="sphere",spangle_type=SPANGLE_SOLID_ROCK,scale=1,seed=1,preset=True)
sps+=[sg]
sg=Spangler(nspangles=nspangles,name="Moon",n_equ=[0,0,1],center_equ=[+3.0,0.0,0.0])
sg.populate_spangler(shape="sphere",spangle_type=SPANGLE_ATMOSPHERIC,scale=0.3,seed=1,preset=True)
sps+=[sg]

sg=Spangler(spanglers=sps)

sg.set_luz(nvec=sci.direction(10,0))
sg.update_illumination_state()

sg.set_observer(nvec=sci.direction(30,20))
sg.update_visibility_state()

sg.plot2d(center_at="Ring")

You may notice that the spangles that are not visible, does not appear in this plot.  Additionally the spangles which are not illuminated appears in a dark color.

When you check the spangles in 3d you may also notice the different states:

In [ ]:
sg.plot3d(center_at="Ring")

You may even interact with the system:

In [ ]:
sg._interact_plot2d()

Or animate:

In [ ]:
nspangles=500
sps=[]
sg=Spangler(nspangles=nspangles,name="Parent",n_equ=[0,0,1],center_equ=[-7,0,0])
sg.populate_spangler(shape="sphere",spangle_type=SPANGLE_STELLAR,scale=3,seed=1,preset=1)
sps+=[sg]
sg=Spangler(nspangles=nspangles,name="Ring",n_equ=sci.direction(0,80))
sg.populate_spangler(shape="ring",spangle_type=SPANGLE_GRANULAR,scale=2.5,seed=1,ri=1.5/2.5,boundary=0)
sps+=[sg]
sg=Spangler(nspangles=nspangles,name="Planet",n_equ=[0,0,1])
sg.populate_spangler(shape="sphere",spangle_type=SPANGLE_SOLID_ROCK,scale=1,seed=1,preset=True)
sps+=[sg]
sg=Spangler(nspangles=nspangles,name="Moon",n_equ=[0,0,1],center_equ=[+3.0,0.0,0.0])
sg.populate_spangler(shape="sphere",spangle_type=SPANGLE_ATMOSPHERIC,scale=0.3,seed=1,preset=True)
sps+=[sg]

sg=Spangler(spanglers=sps)

nobs=Plot.calc_flyby(normal=[0,0,1],start=0,stop=360,num=1,lat=30)
nluz=Plot.calc_flyby(normal=[0,0,1],start=0,stop=360,num=5,lat=20)

#anim,dirs=sg._animate_plot2d(nobs=nobs,nluz=nluz,interval=100,center_at="Ring",axis=False,not_plot=["Parent"])
anim,dirs=sg._animate_plot2d(filename="/tmp/flyby-plot2d-luz.gif",nobs=nobs,nluz=nluz,interval=100,center_at="Ring",axis=False)

In [ ]:
nobs=Plot.calc_flyby(normal=[0,0,1],start=0,stop=360,num=1,lat=30)
nluz=Plot.calc_flyby(normal=[0,0,1],start=0,stop=360,num=20,lat=20)

sg.plot2d(newfig=True)
anim,dirs=sg._animate_plot2d(nobs=nobs,nluz=nluz,interval=1000,center_at="Ring",axis=False)

Pryngles compute the direction on which light arrives into a planet:

In [ ]:
sg=Spangler(nspangles=500,n_equ=[0,1,1])
sg.populate_spangler(shape="sphere",scale=3,seed=1)
sg.set_observer(nvec=[0,0,1])
sg.set_luz(nvec=[1,0,0])

coords="obs"
sg.plot2d(coords=coords,show_azim=True)

--- 
*The authors*, Last update: september, 2022.